## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장
+ 작성: 임경호
+ 한국수출입은행 Open API https://www.koreaexim.go.kr/ir/HPHKIR019M01

### 1. 데이터를 가져올 날짜 설정
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [5]:
import requests
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [6]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()

In [7]:
print(date_from, date_to)

20230727 20230903


### 2. API를 활용한 데이터 수집 (한국수출입은행 Open API)

In [8]:
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [9]:
df_fx_all.shape

(598, 11)

In [10]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230727,AED,344.65,351.62,348.14,348,0,0,348,348.14,아랍에미리트 디르함
1,20230727,AUD,855.62,872.91,864.27,864,0,0,864,864.27,호주 달러
2,20230727,BHD,"3,357.95","3,425.78","3,391.87","3,391",0,0,"3,391","3,391.87",바레인 디나르
3,20230727,BND,955.01,974.3,964.66,964,0,0,964,964.66,브루나이 달러
4,20230727,CAD,958.36,977.73,968.05,968,0,0,968,968.05,캐나다 달러
...,...,...,...,...,...,...,...,...,...,...,...
593,20230901,SAR,349.02,356.07,352.55,352,0,0,352,352.55,사우디 리얄
594,20230901,SEK,119.51,121.92,120.72,120,0,0,120,120.72,스웨덴 크로나
595,20230901,SGD,968.71,988.28,978.5,978,0,0,978,978.5,싱가포르 달러
596,20230901,THB,37.38,38.13,37.76,37,0,0,37,37.76,태국 바트


### 3. 데이터 전처리

In [11]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

In [12]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [13]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230727,AED,344.65,351.62,348.14,348,348,348.14
1,20230727,AUD,855.62,872.91,864.27,864,864,864.27
2,20230727,BHD,3357.95,3425.78,3391.87,3391,3391,3391.87
3,20230727,BND,955.01,974.3,964.66,964,964,964.66
4,20230727,CAD,958.36,977.73,968.05,968,968,968.05
...,...,...,...,...,...,...,...,...
593,20230901,SAR,349.02,356.07,352.55,352,352,352.55
594,20230901,SEK,119.51,121.92,120.72,120,120,120.72
595,20230901,SGD,968.71,988.28,978.5,978,978,978.5
596,20230901,THB,37.38,38.13,37.76,37,37,37.76


In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

### 4. 테이블에 데이터 저장 (mdw.weather)

In [15]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()